In [4]:
import pandas as pd
import os
from sklearn import preprocessing
import time

In [5]:
seconds = time.time()
%matplotlib inline

In [6]:
print("This process may take some time (up to 10 minutes) depending on the performance of a computer.\n\n\n")

This process may take some time (up to 10 minutes) depending on the performance of a computer.





In [7]:
number = "0123456789"

In [8]:
# CSV files
csv_files = [
    "Monday-WorkingHours.pcap_ISCX",
    "Tuesday-WorkingHours.pcap_ISCX",
    "Wednesday-WorkingHours.pcap_ISCX",
    "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX",
    "Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX",
    "Friday-WorkingHours-Morning.pcap_ISCX",
    "Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX",
    "Friday-WorkingHours-Afternoon-DDos.pcap_ISCX"
]

In [9]:
# Column headers
main_labels=["Flow ID","Source IP","Source Port","Destination IP","Destination Port","Protocol","Timestamp","Flow Duration","Total Fwd Packets",
   "Total Backward Packets","Total Length of Fwd Packets","Total Length of Bwd Packets","Fwd Packet Length Max","Fwd Packet Length Min",
   "Fwd Packet Length Mean","Fwd Packet Length Std","Bwd Packet Length Max","Bwd Packet Length Min","Bwd Packet Length Mean","Bwd Packet Length Std",
   "Flow Bytes/s","Flow Packets/s","Flow IAT Mean","Flow IAT Std","Flow IAT Max","Flow IAT Min","Fwd IAT Total","Fwd IAT Mean","Fwd IAT Std","Fwd IAT Max",
   "Fwd IAT Min","Bwd IAT Total","Bwd IAT Mean","Bwd IAT Std","Bwd IAT Max","Bwd IAT Min","Fwd PSH Flags","Bwd PSH Flags","Fwd URG Flags","Bwd URG Flags",
   "Fwd Header Length","Bwd Header Length","Fwd Packets/s","Bwd Packets/s","Min Packet Length","Max Packet Length","Packet Length Mean","Packet Length Std",
   "Packet Length Variance","FIN Flag Count","SYN Flag Count","RST Flag Count","PSH Flag Count","ACK Flag Count","URG Flag Count","CWE Flag Count",
   "ECE Flag Count","Down/Up Ratio","Average Packet Size","Avg Fwd Segment Size","Avg Bwd Segment Size","faulty-Fwd Header Length","Fwd Avg Bytes/Bulk",
   "Fwd Avg Packets/Bulk","Fwd Avg Bulk Rate","Bwd Avg Bytes/Bulk","Bwd Avg Packets/Bulk","Bwd Avg Bulk Rate","Subflow Fwd Packets","Subflow Fwd Bytes",
   "Subflow Bwd Packets","Subflow Bwd Bytes","Init_Win_bytes_forward","Init_Win_bytes_backward","act_data_pkt_fwd",
   "min_seg_size_forward","Active Mean","Active Std","Active Max","Active Min","Idle Mean","Idle Std","Idle Max","Idle Min","Label","External IP"]

In [10]:
main_labels2 = main_labels
main_labels = (",".join(i for i in main_labels))
main_labels = main_labels + "\n"
flag = True

In [11]:
for i in range(len(csv_files)):
    fp = open(str(i)+".csv", "w")
    fp.write(main_labels)
    with open("./CSVs/"+csv_files[i]+".csv", "r") as file:
        while True:
            try:
                line = file.readline()
                if line[0] in number:
                    if " – " in str(line): ##  if there is "–" character ("–", Unicode code:8211) in the flow ,  it will be chanced with "-" character ( Unicode code:45).
                        line = str(line).replace(" – "," - ")
                    line = str(line).replace("inf", "0")
                    line = str(line).replace("Infinity", "0")
                    line = str(line).replace("NaN", "0")

                    fp.write(str(line))
                else:
                    continue
            except:
                break
    fp.close()

In [12]:
df = pd.read_csv(str(i)+".csv",low_memory=False)
df = df.fillna(0)

In [13]:
string_features = [
    "Flow Bytes/s",
    "Flow Packets/s"
]
for ii in string_features:
    # some data in the "Flow Bytes/s" and "Flow Packets/s" columns are not numeric. This loops purpose is to fix this bug
    df[ii] = df[ii].replace("Infinity", -1)
    df[ii] = df[ii].replace("NaN", 0)
    number_or_not = []
    for iii in df[ii]:
        try:
            k = int(float(iii))
            number_or_not.append(int(k))
        except:
            number_or_not.append(iii)
    df[ii] = number_or_not

In [14]:
string_features = []
for j in main_labels2:
    # loop for detection of non-numeric properties (columns)
    if df[j].dtype == "object":
        string_features.append(j)


In [15]:
try:
    string_features.remove("Label") # The "Label" property was removed from the list, because it has to remain "categorical" for using with different machine learning approach
except:
    print("Error") # TODO: add more specific statement here
labelencoder_X = preprocessing.LabelEncoder()